#### C-1.13
Write a pseudo-code description of a function that reverses a list of n integers, so that the numbers are listed in the opposite order than they were before, and compare this method to an equivalent Python function for doing the same thing.

In [260]:
def my_reverse(arr):
    rev_arr = []
    i = -1
    while i > -len(arr)-1:
        rev_arr.append(arr[i])
        i = i - 1
    return rev_arr
my_reverse([1,2,3,4,5])

# there is also the slice option with arr[::-1]


[5, 4, 3, 2, 1]

I am not entirely sure where to find source code for Python builtin functions. The suggested method is to use **inspect.getsource()**, but it does not appear to work on builtins.

That being said, Python itself is implemented in C, and that code can be found here https://github.com/python/cpython. If we dig around the **Object/listobject.c** file, we find a function whose purpose appers to be to reverse a list (see cell below).

The principal differences compared to my implementation are as follows:

* my implementation - creates an empty list and populates it by iterating the original list in reverse order

* C implementation - traverses the list from both ends simultaneously until the indecies in the middle (hi--, lo++). At each iteration step, the function sets the **lo** element to **hi**, and **hi** to **lo** (via temp var **t**). The C implementation uses less memory, and takes half as many steps to complete.


In [1]:
#
#/* Reverse a slice of a list in place, from lo up to (exclusive) hi. */
# static void
# reverse_slice(PyObject **lo, PyObject **hi)
# {
#    assert(lo && hi);
#
#   --hi;
#   while (lo < hi) {
#        PyObject *t = *lo;
#        *lo = *hi;
#        *hi = t;
#        ++lo;
#        --hi;
#    }
#}

# For clarity, I rewrote the C implementation in Python:
arr = [1,2,3,4,5]
lo, hi = 0, len(arr)-1
while lo < hi:
    arr[lo], arr[hi] = arr[hi], arr[lo]
    lo = lo + 1
    hi = hi - 1
print(arr)

[5, 4, 3, 2, 1]


#### C-1.14
Write a short Python function that takes a sequence of integer values and determines if there is a distinct pair of numbers in the sequence whose product is odd.

In [33]:
def is_odd(arr):
    odd = []
    for i, vi in enumerate(arr):
        for vj in arr[i+1:]:
            if (vi*vj) % 2 == 1:
                odd.append((vi,vj))
    return odd

# Outer product version using numpy
import numpy as np
def is_odd2(arr):
    mat = np.outer(arr, arr)
    odd = (mat % 2 == 1).nonzero()
    odd_idx = []
    for x,y in zip(*odd):
        if x!=y and (y,x) not in odd_idx:
            odd_idx.append((x,y))
    return odd_idx

# Faster two-loop
def is_odd3(arr):
    odd = []
    odd_pair = []
    for i, vi in enumerate(arr):
        if vi%2 == 1:
            for j in odd:
                odd_pair.append((i,j))
            odd.append(i)
    return odd_pair

"""
In [19]: %timeit is_odd(range(0,10))
8.31 µs ± 120 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

In [20]: %timeit is_odd2(range(0,10))
35.7 µs ± 405 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

In [21]: %timeit is_odd3(range(0,10))
3.13 µs ± 52.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
"""

is_odd(range(0,5))

[(1, 3)]

#### C-1.15
Write a Python function that takes a sequence of numbers and determines if all the numbers are different from each other (that is, they are distinct).

In [6]:
def is_distinct(arr):
    if len(arr) == len(set(arr)):
        return True
    else:
        return False

print("is %s distinct? %s " % ([1,2,3], is_distinct([1,2,3])))
print("is %s distinct? %s " % ([1,2,3,3], is_distinct([1,2,3,3])))

is [1, 2, 3] distinct? True 
is [1, 2, 3, 3] distinct? False 


#### C-1.16
In our implementation of the scale function (page25),the body of the loop executes the command data[j] = factor. We have discussed that numeric types are immutable, and that use of the   = operator in this context causes the creation of a new instance (not the mutation of an existing instance). How is it still possible, then, that our implementation of scale changes the actual parameter sent by the caller?

Scaled data[j] does not change the orignal value. Instead it points to a new object at a new memory address (see cell below).

I don't know what happens to the orignal object referenced by data[j]. I assume it's cleared from memory. Nevertheless, we do not mutate it.

In [9]:
def scale(data, factor):
    for j in range(len(data)):
        print("memory address old: %s" % id(data[j]))
        data[j] *= factor
        print("memory address new: %s" % id(data[j]))

scale([1], 2)

#Whenever you spawn a unique value for the first time, it's assigned to 
#an address in memory, and it appears to persist there indefinetely (?)
#(wouldnt this lead to memory issues?).
#So if you set x = 10, every other '10' in your program (such the '10' in n**10) 
#will reference the same object in memory as the original '10' from 'x=10'

# scale([1], 1)
# x = 10**1
# print(id(x))
# print(id(10))


memory address old: 4403573872
memory address new: 4403573904


#### C-1.17
Had we implemented the scale function (page 25) as follows, does it work properly? Explain why or why not.

No, it wouldn't work, because **val** is a new and independent object within the for-loop namespace.

In [10]:
def scale(data, factor):
    print(id(data[0]))
    for val in data:
        print(id(val))
        val *= factor
        print(id(val))
    print(data)

scale([1],2)

4403573872
4403573872
4403573904
[1]


#### C-1.18
Demonstrate how to use Python’s list comprehension syntax to produce
the list [0, 2, 6, 12, 20, 30, 42, 56, 72, 90].

In [135]:
[i*(i+1) for i in range(10)]

[0, 2, 6, 12, 20, 30, 42, 56, 72, 90]

#### C-1.19
Demonstrate how to use Python’s list comprehension syntax to produce the list [a , b , c ,..., z], but without having to type all 26 such characters literally.

In [151]:
arr = [chr(i) for i in range(97,123)]
print(arr)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


#### C-1.20
Python’s random module includes a function shuffle(data) that accepts a list of elements and randomly reorders the elements so that each possible order occurs with equal probability. The random module includes a more basic function randint(a, b) that returns a uniformly random integer from a to b (including both endpoints). Using only the randint function, implement your own version of the shuffle function.

In [37]:
from random import randint

def my_shuffle(data):
    shuffled = []
    while len(data) > 0:
        i = randint(0,len(data)-1)
        shuffled.append(data.pop(i))
    return shuffled

data = list(range(10))
print(data)
for i in range(5):
    data = my_shuffle(data)
    print(data)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 3, 1, 6, 7, 4, 5, 8, 2, 9]
[3, 4, 9, 7, 1, 2, 6, 8, 5, 0]
[7, 8, 5, 9, 0, 6, 2, 3, 1, 4]
[1, 6, 2, 0, 5, 8, 4, 9, 3, 7]
[1, 3, 0, 6, 7, 8, 4, 9, 5, 2]


#### C-1.21
Write a Python program that repeatedly reads lines from standard input until an EOFError is raised, and then outputs those lines in reverse order (a user can indicate end of input by typing ctrl-D).

In [41]:
# This one does not work in Jupyter Notebook
def read_lines():
    r = []
    while True:
        try:
            r.append(input("Type something: "))
        except EOFError:
            return r[::-1]
print(read_lines())

Type something: a
Type something: b
Type something: c
Type something: 
Type something: 
Type something: 


KeyboardInterrupt: 

#### C-1.22
Write a short Python program that takes two arrays a and b of length n
storing int values, and returns the dot product of a and b. That is, it returns
an array c of length n such that c[i] = a[i] · b[i], for i = 0,...,n-1

In [11]:
# def dot_product(a,b):
#     c = []
#     for i in range(len(a)):
#         c.append(a[i]*b[i])
#     return c

# List comprehension version
def dot_product(a,b):
    return [x*y for x,y in zip(a,b)]

a, b = range(10), range(10)
dot_product(a,b)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

#### C-1.23
Give an example of a Python code fragment that attempts to write an ele- ment to a list based on an index that may be out of bounds. If that index is out of bounds, the program should catch the exception that results, and print the following error message:
“Don’t try buffer overflow attacks in Python!”

In [193]:
import sys

a = [1]
try:
    a[1] = 0
except IndexError:
    print("Don’t try buffer overflow attacks in Python!")
    raise
#except:
#    print(sys.exc_info())
#    raise

Don’t try buffer overflow attacks in Python!


IndexError: list assignment index out of range

#### C-1.24
Write a short Python function that counts the number of vowels in a given
character string.

In [215]:
VOWELS = ['a', 'e', 'i', 'o', 'u']

#
def count_vowel(string):
    count = 0
    for char in string:
        if char in VOWELS:
            count += 1
    return count

s='abcdaa'
print(count_vowel(s))

# same logic in list comprehension form
print( sum([i in VOWELS for i in s]) )

3
3


#### C-1.25
Write a short Python function that takes a strings, represent in a sentence, and returns a copy of the string with all punctuation removed. For example, if given the string "Let's try, Mike.", this function would return "Lets try Mike".

In [241]:
import re
s = "Let's try, Mike."

re.sub(r'[^\w\s]', '', s) # not(^) word char (\w) or space (\s)

'Lets try Mike'

On use of caret (^) within regex set (that would be the brackets [...]):

Characters that are not within a range can be matched by complementing the set. If the first character of the set is '^', all the characters that are not in the set will be matched. For example, [^5] will match any character except '5'.
https://docs.python.org/2/library/re.html

#### C-1.26
Write a short program that takes as input three integers, a, b, and c, from the console and determines if they can be used in a correct arithmetic formula (in the given order), like “a+b = c,” “a = b−c,” or “a∗b = c.”

In [18]:
def formula():
    a=float(input("Enter the first number:"))
    b=float(input("Enter the second number:"))
    c=float(input("Enter the third number:"))
    if a+b==c:
        return True
    elif a-b==abs(c): # a-b == c | a == b-c
        return True
    elif a*b==c:
        return True
    elif a==b*c: # eq a/b==c
        return True
    elif a*c==b: # eq a==b/c; mutiplication is faster than division
        return True
    elif a%b==c:
        return True
    elif a==b%c:
        return True
    else:
        return False
formula()

Enter the first number:3
Enter the second number:4
Enter the third number:0


#### C-1.27
In Section 1.8, we provided three different implementations of a generator that computes factors of a given integer. The third of those implementations, from page 41, was the most efficient, but we noted that it did not yield the factors in increasing order. Modify the generator so that it reports factors in increasing order, while maintaining its general performance advantages.

In [24]:
def factors(n):
    k = 1
    tmp = []
    while k * k < n:
        if n % k == 0:
            yield k
            # save for later to yield
            tmp.insert(0, n // k) 
        k += 1
    if k * k == n: 
        yield k
    for x in tmp:
        yield x
print(list(factors(81)))

[1, 3, 9, 27, 81]


#### C-1.28
Give an implementation of a function named norm such that norm(v, p) returns the p-norm value of v and norm(v) returns the Euclidean norm of v. You may assume that v is a list of numbers.

In [26]:
def norm(v, p=2):
    return sum([i**p for i in v])**(1./p)

v = list(range(10))

print( norm(v,3) )
print( norm(v) )

12.651489979526238
16.881943016134134
